# Feature Extraction and Data Preparation

<div class="span5 alert alert-info">
    Applying Feature Extraction and Data preparation for Model Building
</div>

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load data
# Cust Offer History
hist = pd.read_csv("../../data/trans_Hist16000.csv")

# Offer Information
offers = pd.read_csv("../../data/offers.csv")

# Transactions for customer who received Offer
trans = pd.read_csv("../../data/transactions_16000.csv")

In [3]:
hist_offer = hist.merge(offers,left_on='offer',right_on='offer')
replace_repeater = {"repeater":     {"t": 1, "f": 0}}
hist_offer.replace(replace_repeater,inplace=True)
hist_offer.head()

,id,chain,offer,market,repeattrips,repeater,offerdate,category,quantity,company,offervalue,brand
0,705936120,165,1208329,21,0,0,2013-04-23,2119,1,108079383,1.0,6926
1,3814736505,77,1208329,37,2,1,2013-04-22,2119,1,108079383,1.0,6926
2,4627974746,153,1208329,12,10,1,2013-04-23,2119,1,108079383,1.0,6926
3,4623268450,153,1208329,12,0,0,2013-04-25,2119,1,108079383,1.0,6926
4,501138510,21,1208329,10,0,0,2013-04-22,2119,1,108079383,1.0,6926


In [4]:
trans['total_transactions_count'] = ''
trans['total_purchase_category_count']=''
trans['total_purchase_company_count']=''
trans['total_purchase_brand_count']=''
trans['time_period']=''

# Transactions Aggregation by ID
agg_trans = pd.DataFrame(trans.groupby(['id']).agg({'total_transactions_count':"count",'purchaseamount':sum,'purchasequantity':sum})).reset_index()
agg_trans.head()

,id,purchaseamount,purchasequantity,total_transactions_count
0,13089312,3890.68,1469,1218
1,13873775,6088.08,2955,1463
2,16138642,1872.07,734,537
3,16238376,1836.97,722,576
4,17486592,8078.55,2014,1526


In [5]:
# Transactions Aggregation by CATEGORY, COMPANY, BRAND
agg_trans_cat = pd.DataFrame(trans.groupby(['id','category']).agg({'total_purchase_category_count':"count"})).reset_index().fillna(0)
agg_trans_comp = pd.DataFrame(trans.groupby(['id','company']).agg({'total_purchase_company_count':"count"})).reset_index().fillna(0)
agg_trans_brand = pd.DataFrame(trans.groupby(['id','brand']).agg({'total_purchase_brand_count':"count"})).reset_index().fillna(0)

In [6]:
k1 = pd.merge(hist_offer[['id','offer','repeater','repeattrips','company','category','brand']], agg_trans_comp,on=('id','company'),how='left').fillna(0)
k2 = pd.merge(k1, agg_trans_cat,on=('id','category'),how='left').fillna(0)
k3 = pd.merge(k2, agg_trans_brand,on=('id','brand'),how='left').fillna(0)
k4 = pd.merge(agg_trans,k3 ,on=('id'),how='left').fillna(0)
k4.fillna(0,inplace=True)
k4.head()

,id,purchaseamount,purchasequantity,total_transactions_count,offer,repeater,repeattrips,company,category,brand,total_purchase_company_count,total_purchase_category_count,total_purchase_brand_count
0,13089312,3890.68,1469,1218,1204821,0,0,107717272,5619,102504,3.0,0.0,3.0
1,13873775,6088.08,2955,1463,1197502,0,0,106414464,3203,13474,16.0,0.0,0.0
2,16138642,1872.07,734,537,1204576,0,0,104610040,5616,15889,2.0,0.0,2.0
3,16238376,1836.97,722,576,1197502,1,1,106414464,3203,13474,0.0,5.0,0.0
4,17486592,8078.55,2014,1526,1197502,1,1,106414464,3203,13474,0.0,1.0,0.0


In [7]:
# Offer Date Based Transaction Aggregation
trans = pd.merge(hist_offer[['id','offerdate']], trans,on=('id')).fillna(0)
trans['hr']=0
trans['cnt']=1

trans['date'] = pd.to_datetime(trans['date'])
trans['date'] += pd.to_timedelta(trans['hr'], unit='h')

trans['offerdate'] = pd.to_datetime(trans['offerdate'])
trans['offerdate'] += pd.to_timedelta(trans['hr'], unit='h')

trans.index = trans['date']

trans['time_period'] = ((trans.offerdate - trans.date)/np.timedelta64(1, 'M'))

trans['time_period'] = trans['time_period'].astype(int)

trans_cat_sum=trans.groupby(['id','category','time_period']).sum().reset_index()

del trans_cat_sum['index']
del trans_cat_sum['chain']
del trans_cat_sum['company']
del trans_cat_sum['dept']
del trans_cat_sum['brand']
del trans_cat_sum['productsize']
del trans_cat_sum['hr']

trans_cat_sum.head()

,id,category,time_period,purchasequantity,purchaseamount,cnt
0,13089312,0,6,1,2.99,1
1,13089312,0,7,1,3.29,1
2,13089312,0,8,2,5.98,2
3,13089312,0,10,1,3.59,1
4,13089312,202,5,2,3.98,1


In [8]:
trans_mntly_detls = trans_cat_sum[trans_cat_sum['time_period']<1]
trans_monthly_summary = trans_mntly_detls.groupby(['id']).sum().reset_index()
trans_2_mnts_detls = trans_cat_sum[trans_cat_sum['time_period']<2]
trans_2_mnts_summary = trans_2_mnts_detls.groupby(['id']).sum().reset_index()
trans_qtly_detls = trans_cat_sum[trans_cat_sum['time_period']<3]
trans_qtly_summary = trans_qtly_detls.groupby(['id']).sum().reset_index()
trans_half_yr_detls = trans_cat_sum[trans_cat_sum['time_period']<6]
trans_half_yr_summary = trans_half_yr_detls.groupby(['id']).sum().reset_index()
trans_9_mnts_detls = trans_cat_sum[trans_cat_sum['time_period']<9]
trans_9_mnts_summary = trans_9_mnts_detls.groupby(['id']).sum().reset_index()
trans_12_mnts_detls = trans_cat_sum[trans_cat_sum['time_period']<12]
trans_12_mnts_summary = trans_12_mnts_detls.groupby(['id']).sum().reset_index()

trans_monthly_summary = trans_monthly_summary[['id','purchaseamount']]
trans_2_mnts_summary = trans_2_mnts_summary[['id','purchaseamount']]
trans_qtly_summary = trans_qtly_summary[['id','purchaseamount']]
trans_half_yr_summary = trans_half_yr_summary[['id','purchaseamount']]
trans_9_mnts_summary = trans_9_mnts_summary[['id','purchaseamount']]
trans_12_mnts_summary = trans_12_mnts_summary[['id','purchaseamount']]

trans_monthly_summary.columns = ['id','1_month_total_purchase_amt']
trans_2_mnts_summary.columns = ['id','2_mnths_total_purchase_amt']
trans_qtly_summary.columns = ['id','3_mnths_total_purchase_amt']
trans_half_yr_summary.columns = ['id','6_mnths_total_purchase_amt']
trans_9_mnts_summary.columns = ['id','9_mnths_total_purchase_amt']
trans_12_mnts_summary.columns = ['id','12_mnths_total_purchase_amt']

k5 = pd.merge(trans_monthly_summary, k4,on=('id'),how='right').fillna(0)
k5.head()

,id,1_month_total_purchase_amt,purchaseamount,purchasequantity,total_transactions_count,offer,repeater,repeattrips,company,category,brand,total_purchase_company_count,total_purchase_category_count,total_purchase_brand_count
0,13089312,376.00,3890.68,1469,1218,1204821,0,0,107717272,5619,102504,3.0,0.0,3.0
1,13873775,500.59,6088.08,2955,1463,1197502,0,0,106414464,3203,13474,16.0,0.0,0.0
2,16138642,70.99,1872.07,734,537,1204576,0,0,104610040,5616,15889,2.0,0.0,2.0
3,16238376,37.45,1836.97,722,576,1197502,1,1,106414464,3203,13474,0.0,5.0,0.0
4,17486592,752.90,8078.55,2014,1526,1197502,1,1,106414464,3203,13474,0.0,1.0,0.0


In [9]:
k6 = pd.merge(trans_qtly_summary, k5,on=('id'),how='right').fillna(0)
k7 = pd.merge(trans_half_yr_summary, k6,on=('id'),how='right').fillna(0)
k8 = pd.merge(trans_9_mnts_summary, k7,on=('id'),how='right').fillna(0)
k9 = pd.merge(trans_12_mnts_summary, k8,on=('id'),how='right').fillna(0)

In [10]:
trans_summary_avg = trans_cat_sum.groupby(['id']).mean().reset_index()
trans_summary_avg = trans_summary_avg[['id','purchaseamount']]
trans_summary_avg.columns = ['id','total_trans_purchaseamount_avg']
k10 = pd.merge(trans_summary_avg, k9,on=('id'),how='right').fillna(0)

In [11]:
trans_summary_avg_cat = trans_cat_sum.groupby(['id','category']).mean().reset_index()
trans_summary_avg_cat = trans_summary_avg_cat[['id','category','purchaseamount']]
trans_summary_avg_cat.columns = ['id','category','purchaseamount_category_avg']
k11 = pd.merge(k10, trans_summary_avg_cat,on=(['id','category']),how='left').fillna(0)

In [12]:
hist_offer['hr']=0
hist_offer['offerdate'] = pd.to_datetime(hist_offer['offerdate'])
hist_offer['offerdate'] += pd.to_timedelta(hist_offer['hr'], unit='h')
hist_offer['offeredmonth'] = pd.DatetimeIndex(hist_offer['offerdate']).month

k12 = pd.merge(k11, hist_offer[['id','offervalue','offeredmonth','quantity']],on=(['id']),how='left').fillna(0)
k12.head()

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,purchaseamount,purchasequantity,total_transactions_count,...,company,category,brand,total_purchase_company_count,total_purchase_category_count,total_purchase_brand_count,purchaseamount_category_avg,offervalue,offeredmonth,quantity
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,107717272,5619,102504,3.0,0.0,3.0,0.00,1.50,4,1
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,106414464,3203,13474,16.0,0.0,0.0,0.00,0.75,3,1
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,104610040,5616,15889,2.0,0.0,2.0,0.00,1.00,4,1
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,106414464,3203,13474,0.0,5.0,0.0,3.41,0.75,4,1
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,106414464,3203,13474,0.0,1.0,0.0,3.49,0.75,4,1


In [13]:
#Category

trans_mntly_detls['category_purchased_count_30days']=1
trans_mntly_detls['category_purchased_amt_30days']=1
trans_mntly_detls['category_purchased_qty_30days']=1
p = trans_mntly_detls.groupby(['id','category']).sum().reset_index()
k13 = pd.merge(k12, p[['id','category','purchaseamount','purchasequantity']],on=(['id','category']),how='left').fillna(0)
#k13.columns = ['id','category','category_purchased_30days']
k13.rename(index=str, columns={'purchaseamount_y':'category_purchased_amt_30days','purchasequantity_y':'category_purchased_qty_30days','purchaseamount_x':'purchaseamount','purchasequantity_x':'purchasequantity'},inplace=True)
k13.head()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,purchaseamount,purchasequantity,total_transactions_count,...,brand,total_purchase_company_count,total_purchase_category_count,total_purchase_brand_count,purchaseamount_category_avg,offervalue,offeredmonth,quantity,category_purchased_amt_30days,category_purchased_qty_30days
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,102504,3.0,0.0,3.0,0.00,1.50,4,1,0.0,0.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,13474,16.0,0.0,0.0,0.00,0.75,3,1,0.0,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,15889,2.0,0.0,2.0,0.00,1.00,4,1,0.0,0.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,13474,0.0,5.0,0.0,3.41,0.75,4,1,0.0,0.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,13474,0.0,1.0,0.0,3.49,0.75,4,1,0.0,0.0


In [14]:
trans_qtly_detls['category_purchased_count_90days']=1
trans_qtly_detls['category_purchased_amt_90days']=1
trans_qtly_detls['category_purchased_qty_90days']=1
p1 = trans_qtly_detls.groupby(['id','category']).sum().reset_index()
k14 = pd.merge(k13, p1[['id','category','purchaseamount','purchasequantity']],on=(['id','category']),how='left').fillna(0)
#k14.columns = ['id','category','category_purchased_90days']
k14.rename(index=str, columns={'purchaseamount_y':'category_purchased_amt_90days','purchasequantity_y':'category_purchased_qty_90days','purchaseamount_x':'purchaseamount','purchasequantity_x':'purchasequantity'},inplace=True)
k14.head()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,purchaseamount,purchasequantity,total_transactions_count,...,total_purchase_category_count,total_purchase_brand_count,purchaseamount_category_avg,offervalue,offeredmonth,quantity,category_purchased_amt_30days,category_purchased_qty_30days,category_purchased_amt_90days,category_purchased_qty_90days
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,0.0,3.0,0.00,1.50,4,1,0.0,0.0,0.00,0.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,0.0,0.0,0.00,0.75,3,1,0.0,0.0,0.00,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,0.0,2.0,0.00,1.00,4,1,0.0,0.0,0.00,0.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,5.0,0.0,3.41,0.75,4,1,0.0,0.0,4.48,2.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,1.0,0.0,3.49,0.75,4,1,0.0,0.0,0.00,0.0


In [15]:
trans_half_yr_detls['category_purchased_count_180days']=1
trans_half_yr_detls['category_purchased_amt_180days']=1
trans_half_yr_detls['category_purchased_qty_180days']=1
p2 = trans_half_yr_detls.groupby(['id','category']).sum().reset_index()
k15 = pd.merge(k14, p2[['id','category','purchaseamount','purchasequantity']],on=(['id','category']),how='left').fillna(0)
#k15.columns = ['id','category','category_purchased_180days']
k15.rename(index=str, columns={'purchaseamount_y':'category_purchased_amt_180days','purchasequantity_y':'category_purchased_qty_180days','purchaseamount_x':'purchaseamount','purchasequantity_x':'purchasequantity'},inplace=True)
k15.head()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,purchaseamount,purchasequantity,total_transactions_count,...,purchaseamount_category_avg,offervalue,offeredmonth,quantity,category_purchased_amt_30days,category_purchased_qty_30days,category_purchased_amt_90days,category_purchased_qty_90days,category_purchased_amt_180days,category_purchased_qty_180days
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,0.00,1.50,4,1,0.0,0.0,0.00,0.0,0.00,0.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,0.00,0.75,3,1,0.0,0.0,0.00,0.0,0.00,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,0.00,1.00,4,1,0.0,0.0,0.00,0.0,0.00,0.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,3.41,0.75,4,1,0.0,0.0,4.48,2.0,4.48,2.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,3.49,0.75,4,1,0.0,0.0,0.00,0.0,0.00,0.0


In [16]:
trans_9_mnts_detls['category_purchased_count_270days']=1
trans_9_mnts_detls['category_purchased_amt_270days']=1
trans_9_mnts_detls['category_purchased_qty_270days']=1
p3 = trans_9_mnts_detls.groupby(['id','category']).sum().reset_index()
k16 = pd.merge(k15, p3[['id','category','purchaseamount','purchasequantity']],on=(['id','category']),how='left').fillna(0)
#k16.columns = ['id','category','category_purchased_270days']
k16.rename(index=str, columns={'purchaseamount_y':'category_purchased_amt_270days','purchasequantity_y':'category_purchased_qty_270days','purchaseamount_x':'purchaseamount','purchasequantity_x':'purchasequantity'},inplace=True)
k16.head()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,purchaseamount,purchasequantity,total_transactions_count,...,offeredmonth,quantity,category_purchased_amt_30days,category_purchased_qty_30days,category_purchased_amt_90days,category_purchased_qty_90days,category_purchased_amt_180days,category_purchased_qty_180days,category_purchased_amt_270days,category_purchased_qty_270days
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,4,1,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,3,1,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,4,1,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,4,1,0.0,0.0,4.48,2.0,4.48,2.0,4.48,2.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,4,1,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0


In [17]:
# Company
trans['cnt']=1
trans_comp_sum=trans.groupby(['id','company','time_period']).sum().reset_index()

del trans_comp_sum['index']
del trans_comp_sum['chain']
del trans_comp_sum['category']
del trans_comp_sum['dept']
del trans_comp_sum['brand']
del trans_comp_sum['productsize']
del trans_comp_sum['hr']

###
trans_mntly_detls = trans_comp_sum[trans_comp_sum['time_period']<1]#trans_summary.groupby(['id']).head(1)
trans_monthly_summary = trans_mntly_detls.groupby(['id']).sum().reset_index()#trans_mntly_detls.groupby(['id']).sum().reset_index()
trans_qtly_detls = trans_comp_sum[trans_comp_sum['time_period']<3]#trans_summary.groupby(['id']).head(3)
trans_qtly_summary = trans_qtly_detls.groupby(['id']).sum().reset_index()#trans_qtly_detls.groupby(['id']).sum().reset_index()
trans_half_yr_detls = trans_comp_sum[trans_comp_sum['time_period']<6]#trans_summary.groupby(['id']).head(6)
trans_half_yr_summary = trans_half_yr_detls.groupby(['id']).sum().reset_index()#trans_half_yr_detls.groupby(['id']).sum().reset_index()
trans_9_mnts_detls = trans_comp_sum[trans_comp_sum['time_period']<9]#trans_summary.groupby(['id']).head(9)
trans_9_mnts_summary = trans_9_mnts_detls.groupby(['id']).sum().reset_index()#trans_9_mnts_detls.groupby(['id']).sum().reset_index()
trans_12_mnts_detls = trans_comp_sum[trans_comp_sum['time_period']<12]#trans_summary.groupby(['id']).head(12)
trans_12_mnts_summary = trans_12_mnts_detls.groupby(['id']).sum().reset_index()#trans_12_mnts_detls.groupby(['id']).sum().reset_index()

###

trans_mntly_detls['company_purchased_count_30days']=1
trans_mntly_detls['company_purchased_amt_30days']=1
trans_mntly_detls['company_purchased_qty_30days']=1

p4 = trans_mntly_detls.groupby(['id','company']).sum().reset_index()
k17 = pd.merge(k16, p4[['id','company','purchaseamount','purchasequantity']],on=(['id','company']),how='left').fillna(0)
k17.rename(index=str, columns={'purchaseamount_y':'company_purchased_amt_30days','purchasequantity_y':'company_purchased_qty_30days','purchaseamount_x':'purchaseamount','purchasequantity_x':'purchasequantity'},inplace=True)

trans_qtly_detls['company_purchased_count_90days']=1
trans_qtly_detls['company_purchased_amt_90days']=1
trans_qtly_detls['company_purchased_qty_90days']=1

p5 = trans_qtly_detls.groupby(['id','company']).sum().reset_index()
k18 = pd.merge(k17, p5[['id','company','purchaseamount','purchasequantity']],on=(['id','company']),how='left').fillna(0)
#k18.columns = ['id','company','company_purchased_90days']
k18.rename(index=str, columns={'purchaseamount_y':'company_purchased_amt_90days','purchasequantity_y':'company_purchased_qty_90days','purchaseamount_x':'purchaseamount','purchasequantity_x':'purchasequantity'},inplace=True)

trans_half_yr_detls['company_purchased_count_180days']=1
trans_half_yr_detls['company_purchased_amt_180days']=1
trans_half_yr_detls['company_purchased_qty_180days']=1

p6 = trans_half_yr_detls.groupby(['id','company']).sum().reset_index()
k19 = pd.merge(k18, p6[['id','company','purchaseamount','purchasequantity']],on=(['id','company']),how='left').fillna(0)
#k19.columns = ['id','company','company_purchased_180days']
k19.rename(index=str, columns={'purchaseamount_y':'company_purchased_amt_180days','purchasequantity_y':'company_purchased_qty_180days','purchaseamount_x':'purchaseamount','purchasequantity_x':'purchasequantity'},inplace=True)

trans_9_mnts_detls['company_purchased_count_270days']=1
trans_9_mnts_detls['company_purchased_amt_270days']=1
trans_9_mnts_detls['company_purchased_qty_270days']=1

p7 = trans_9_mnts_detls.groupby(['id','company']).sum().reset_index()
k20 = pd.merge(k19, p7[['id','company','purchaseamount','purchasequantity']],on=(['id','company']),how='left').fillna(0)
#k20.columns = ['id','company','company_purchased_270days']
k20.rename(index=str, columns={'purchaseamount_y':'company_purchased_amt_270days','purchasequantity_y':'company_purchased_qty_270days','purchaseamount_x':'purchaseamount','purchasequantity_x':'purchasequantity'},inplace=True)
k20.head()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,purchaseamount,purchasequantity,total_transactions_count,...,category_purchased_amt_270days,category_purchased_qty_270days,company_purchased_amt_30days,company_purchased_qty_30days,company_purchased_amt_90days,company_purchased_qty_90days,company_purchased_amt_180days,company_purchased_qty_180days,company_purchased_amt_270days,company_purchased_qty_270days
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,0.00,0.0,7.98,2.0,19.95,5.0,19.95,5.0,19.95,5.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,0.00,0.0,4.70,5.0,11.18,15.0,38.00,33.0,53.43,48.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,0.00,0.0,0.00,0.0,3.99,1.0,8.98,2.0,8.98,2.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,4.48,2.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0


In [18]:
#Brand
trans['cnt']=1
trans_brand_sum=trans.groupby(['id','brand','time_period']).sum().reset_index()

del trans_brand_sum['index']
del trans_brand_sum['chain']
del trans_brand_sum['company']
del trans_brand_sum['dept']
del trans_brand_sum['category']
del trans_brand_sum['productsize']
del trans_brand_sum['hr']

###
trans_mntly_detls = trans_brand_sum[trans_brand_sum['time_period']<1]#trans_summary.groupby(['id']).head(1)
trans_monthly_summary = trans_mntly_detls.groupby(['id']).sum().reset_index()#trans_mntly_detls.groupby(['id']).sum().reset_index()
trans_qtly_detls = trans_brand_sum[trans_brand_sum['time_period']<3]#trans_summary.groupby(['id']).head(3)
trans_qtly_summary = trans_qtly_detls.groupby(['id']).sum().reset_index()#trans_qtly_detls.groupby(['id']).sum().reset_index()
trans_half_yr_detls = trans_brand_sum[trans_brand_sum['time_period']<6]#trans_summary.groupby(['id']).head(6)
trans_half_yr_summary = trans_half_yr_detls.groupby(['id']).sum().reset_index()#trans_half_yr_detls.groupby(['id']).sum().reset_index()
trans_9_mnts_detls = trans_brand_sum[trans_brand_sum['time_period']<9]#trans_summary.groupby(['id']).head(9)
trans_9_mnts_summary = trans_9_mnts_detls.groupby(['id']).sum().reset_index()#trans_9_mnts_detls.groupby(['id']).sum().reset_index()
trans_12_mnts_detls = trans_brand_sum[trans_brand_sum['time_period']<12]#trans_summary.groupby(['id']).head(12)
trans_12_mnts_summary = trans_12_mnts_detls.groupby(['id']).sum().reset_index()#trans_12_mnts_detls.groupby(['id']).sum().reset_index()

###


trans_mntly_detls['brand_purchased_count_30days']=1
trans_mntly_detls['brand_purchased_amt_30days']=1
trans_mntly_detls['brand_purchased_qty_30days']=1

p5 = trans_mntly_detls.groupby(['id','brand']).sum().reset_index()
k21 = pd.merge(k20, p5[['id','brand','purchaseamount','purchasequantity']],on=(['id','brand']),how='left').fillna(0)
#k21.columns = ['id','company','brand_purchased_30days']
k21.rename(index=str, columns={'purchaseamount_y':'brand_purchased_amt_30days','purchasequantity_y':'brand_purchased_qty_30days','purchaseamount_x':'purchaseamount','purchasequantity_x':'purchasequantity'},inplace=True)

trans_qtly_detls['brand_purchased_90days']=1
trans_qtly_detls['brand_purchased_amt_90days']=1
trans_qtly_detls['brand_purchased_qty_90days']=1

p6 = trans_qtly_detls.groupby(['id','brand']).sum().reset_index()
k22 = pd.merge(k21, p6[['id','brand','purchaseamount','purchasequantity']],on=(['id','brand']),how='left').fillna(0)
#k22.columns = ['id','company','brand_purchased_90days']
k22.rename(index=str, columns={'purchaseamount_y':'brand_purchased_amt_90days','purchasequantity_y':'brand_purchased_qty_90days','purchaseamount_x':'purchaseamount','purchasequantity_x':'purchasequantity'},inplace=True)

trans_half_yr_detls['brand_purchased_180days']=1
trans_half_yr_detls['brand_purchased_amt_180days']=1
trans_half_yr_detls['brand_purchased_qty_180days']=1

p7 = trans_half_yr_detls.groupby(['id','brand']).sum().reset_index()
k23 = pd.merge(k22, p7[['id','brand','purchaseamount','purchasequantity']],on=(['id','brand']),how='left').fillna(0)
#k23.columns = ['id','company','brand_purchased_180days']
k23.rename(index=str, columns={'purchaseamount_y':'brand_purchased_amt_180days','purchasequantity_y':'brand_purchased_qty_180days','purchaseamount_x':'purchaseamount','purchasequantity_x':'purchasequantity'},inplace=True)

trans_9_mnts_detls['brand_purchased_270days']=1
trans_9_mnts_detls['brand_purchased_amt_270days']=1
trans_9_mnts_detls['brand_purchased_qty_270days']=1

p8 = trans_9_mnts_detls.groupby(['id','brand']).sum().reset_index()
k24 = pd.merge(k23, p8[['id','brand','purchaseamount','purchasequantity']],on=(['id','brand']),how='left').fillna(0)
#k24.columns = ['id','company','brand_purchased_270days']
k24.rename(index=str, columns={'purchaseamount_y':'brand_purchased_amt_270days','purchasequantity_y':'brand_purchased_qty_270days','purchaseamount_x':'purchaseamount','purchasequantity_x':'purchasequantity'},inplace=True)
k24.head()

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,purchaseamount,purchasequantity,total_transactions_count,...,company_purchased_amt_270days,company_purchased_qty_270days,brand_purchased_amt_30days,brand_purchased_qty_30days,brand_purchased_amt_90days,brand_purchased_qty_90days,brand_purchased_amt_180days,brand_purchased_qty_180days,brand_purchased_amt_270days,brand_purchased_qty_270days
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,19.95,5.0,7.98,2.0,19.95,5.0,19.95,5.0,19.95,5.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,53.43,48.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,8.98,2.0,0.00,0.0,3.99,1.0,8.98,2.0,8.98,2.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0


In [19]:
trans['purchased_company_category_brand_count']=1
trans_cat_comp_brnd_sum=trans.groupby(['id','company','category','brand']).count().reset_index()
trans_cat_comp_brnd_sum = trans_cat_comp_brnd_sum[['id','company','category','brand','purchased_company_category_brand_count']]

k25 = pd.merge(k24, trans_cat_comp_brnd_sum,on=(['id','company','category','brand']),how='left').fillna(0)

trans['purchased_company_category_count']=1
trans_cat_comp_sum=trans.groupby(['id','company','category']).sum().reset_index()
trans_cat_comp_sum = trans_cat_comp_sum[['id','company','category','purchased_company_category_count']]

k26 = pd.merge(k25, trans_cat_comp_sum,on=(['id','company','category']),how='left').fillna(0)
k26.head()

,id,total_trans_purchaseamount_avg,12_mnths_total_purchase_amt,9_mnths_total_purchase_amt,6_mnths_total_purchase_amt,3_mnths_total_purchase_amt,1_month_total_purchase_amt,purchaseamount,purchasequantity,total_transactions_count,...,brand_purchased_amt_30days,brand_purchased_qty_30days,brand_purchased_amt_90days,brand_purchased_qty_90days,brand_purchased_amt_180days,brand_purchased_qty_180days,brand_purchased_amt_270days,brand_purchased_qty_270days,purchased_company_category_brand_count,purchased_company_category_count
0,13089312,5.279077,3610.73,2431.84,1566.45,892.43,376.00,3890.68,1469,1218,...,7.98,2.0,19.95,5.0,19.95,5.0,19.95,5.0,0.0,0.0
1,13873775,7.352754,5644.72,4209.56,2998.90,1379.47,500.59,6088.08,2955,1463,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0
2,16138642,5.538669,1793.45,1630.18,1358.97,677.12,70.99,1872.07,734,537,...,0.00,0.0,3.99,1.0,8.98,2.0,8.98,2.0,0.0,0.0
3,16238376,4.262111,1513.81,1097.80,547.34,467.67,37.45,1836.97,722,576,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0
4,17486592,8.752492,7028.25,5413.99,4162.00,1802.40,752.90,8078.55,2014,1526,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0


In [20]:
k26.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16000 entries, 0 to 15999
Data columns (total 49 columns):
id                                        16000 non-null int64
total_trans_purchaseamount_avg            16000 non-null float64
12_mnths_total_purchase_amt               16000 non-null float64
9_mnths_total_purchase_amt                16000 non-null float64
6_mnths_total_purchase_amt                16000 non-null float64
3_mnths_total_purchase_amt                16000 non-null float64
1_month_total_purchase_amt                16000 non-null float64
purchaseamount                            16000 non-null float64
purchasequantity                          16000 non-null int64
total_transactions_count                  16000 non-null int64
offer                                     16000 non-null int64
repeater                                  16000 non-null int64
repeattrips                               16000 non-null int64
company                                   16000 non-null i

In [21]:
k26.to_csv("../../data/data.csv", sep=',', encoding='utf-8')